In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numexpr as ne
# import poppy
import timeit
from scipy.special import erfc
from numba import njit

# Propogate Divergence & Waist Rays of Gaussian Beamlets - NONastigmatic beamlets

# Wavefront Parameters
wl = 2.2e-6  # wavelength
OF = 2 # beamlet overlap factor
wo =3*wl
#N  = 50 # Number of beamlets to trace    

# Generate Poppy aperture
amax = 5e-4     # meters
#ap = poppy.CircularAperture(radius=amax)

# Detector Parameters
sample = 512       # number of detector pixels across

# Calculate Gaussian Beamlet Parameters
#wo = amax*OF/(2*N)
N = round(np.pi*(amax*OF/(2*wo)/2)**2)
print(wo,'mm beam waist')
print(wo/wl,'waves beam waist')
print(N,' Number of Beamlets')

zr = np.pi*wo**2/wl # rayleigh range
k  = 2*np.pi/wl # wave number

# System Parameters
d   = 5e-3 # propagation distance
f   = 5e-3    # focal length
n   = 1       # refractive index 

# configure detector dimensions
u   = np.linspace(-amax,amax,sample)
v   = u

# Define a Q Matrix - diagonal zero for nonastigmatic case
qxx = 1/(1j*zr)
qxy = 0
qyx = 0
qyy = 1/(1j*zr)
Q = np.array([[qxx,qxy],
             [qyx,qyy]],dtype='complex') # Defines the matrix of inverse q parameters

print(amax**2/(d*wl),' Fresnel Number')
print(d/(2*amax),' Working F/#')
print(wl/(np.pi*wo)*5e-1,'gaussian beam divergence angle in degrees')

if f <=1000:
    print(1.22*wl*(d/(2*amax)),'Airy Radius')

6.6e-06 mm beam waist
3.0 waves beam waist
4508  Number of Beamlets
22.727272727272723  Fresnel Number
5.0  Working F/#
0.05305164769729845 gaussian beam divergence angle in degrees
1.342e-05 Airy Radius


In [2]:
# Generate fibbonacci spaced list of xy center rays
r = amax
c = np.array([0,0]) # XY offset from a spiral
R = r*np.sqrt(np.linspace(1/2,N-1/2,N))/np.sqrt(N-1/2)
T = 4/(1+np.sqrt(5))*np.pi*np.linspace(1,N,N);
X = c[0] +R*np.cos(T)
Y = c[1] +R*np.sin(T)

#X = np.linspace(-amax,amax,N)
#Y = np.linspace(-amax,amax,N)
#print(X)

In [3]:
# Configure Propagation

# prepare array of base rays
base = np.array([[X],
                 [Y],
                 [0*X],
                 [0*Y]])

# prepare system matrix
focus = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [-1/f,0,1,0],
                  [0,-1/f,0,1]])

propg = np.array([[1,0,d,0],
                 [0,1,0,d],
                 [0,0,1,0],
                 [0,0,0,1]])

system = np.matmul(propg,focus)


prop = (np.matmul(system,base[:,0]))
tbox = np.sqrt((prop[0,:]-base[0,:])**2 + (prop[1,:]-base[1,:])**2 + d**2)
#u,v = np.meshgrid(u,v)
A = system[0:2,0:2]
B = system[0:2,2:4]
C = system[2:4,0:2] # factor of 2 disparity on this matrix
D = system[2:4,2:4]
print(prop)

[[ 8.08320223e-23 -1.65996339e-23 -1.49142215e-22 ... -6.69098131e-21
   1.03183273e-20 -8.52604886e-21]
 [ 7.40487538e-23 -1.89144266e-22  1.94529609e-22 ...  7.96970893e-21
  -1.35707298e-21 -5.96992882e-21]
 [ 7.76608139e-04 -1.59483957e-04 -1.43291055e-03 ... -6.42848020e-02
   9.91351786e-02 -8.19155424e-02]
 [ 7.11436672e-04 -1.81723743e-03  1.86897808e-03 ...  7.65704067e-02
  -1.30383218e-02 -5.73571610e-02]]


In [4]:
# Define Gaussian & Half-Gaussian Beamlet w/Qprop

def Qprop(system,Q): # NON-ASTIGMATIC
    A = system[0:2,0:2]
    B = system[0:2,2:4]
    C = system[2:4,0:2] # factor of 2 disparity on this matrix
    D = system[2:4,2:4]
    
    # Prepare complex curvature matrix
    Qn = (C + np.matmul(D,Q)) # matrix numerator
    Qd = np.linalg.inv(A + np.matmul(B,Q)) # matrix denominator
    Qp = np.matmul(Qn,Qd) # propagated complex curvature
    
    return Qp

def GauBlet(system,Qp,Q,u,uo,v,vo):
    # Set up Electric Field Expression with detector dimensions
    u = u+uo
    v = v+vo
    Gamp = np.exp(-1j*k*d)/np.sqrt(np.linalg.norm(A+np.matmul(B,Q)))
    rQmult = Qp[0,0]*u**2 + (Qp[0,1]+Qp[1,0])*u*v + Qp[1,1]*v**2
    Gphase = np.exp((-1j*k/2)*rQmult)
    Efield = Gamp*Gphase
    
    return Efield

def GauBphase(system,Qp,Q,u,uo,v,vo):
    # Set up Electric Field Expression with detector dimensions
    u = u+uo
    v = v+vo
    Gphase = (-1j*k/2)*(-1j*k*d)*(Qp[0,0]*u**2 + (Qp[0,1]+Qp[1,0])*u*v + Qp[1,1]*v**2)
    
    return Gphase
    

def HGauBlet(system,Qp,Q,u,v):
    
    # Set up Electric Field Expression with detector dimensions
    A = system[0:2,0:2]
    B = system[0:2,2:4]
    u,v = np.meshgrid(u,v)
    Gamp = np.exp(-1j*k*d)/np.sqrt(np.linalg.norm(A+np.matmul(B,Q)))
    rQmult = Qp[0,0]*u**2 + (Qp[0,1]+Qp[1,0])*u*v + Qp[1,1]*v**2
    Gphase = np.exp((-1j*k/2)*rQmult)
    Hgaubamp = .5*erfc(-np.sqrt((1j*k)/(2*B[0,0]*(A[0,0] + B[0,0]*Q[0,0])))*u)
    Efield = Gamp*Gphase*Hgaubamp
    Ifield = np.real(Efield*np.conj(Efield))
    
    return Ifield

In [5]:
#Qp = Qprop(system,Q)
#print(Qp)
#Dfield = np.zeros([sample,sample,N],dtype='complex')
#Qp = Qprop(system,Q)
# try populate a circular aperture with beamlets

#@njit
#def dphase_loop(N,Dfield,system,Qp,Q,u,X,v,Y):
#    for ind in range(N):
#        uo = X[ind]
#        vo = Y[ind]
#        u = u+uo
#        v = v+vo
#        Gphase = (-1j*k/2)*(-1j*k*d)*(Qp[0,0]*u**2 + (Qp[0,1]+Qp[1,0])*u*v + Qp[1,1]*v**2)
#        Dfield[:,:,ind] = Gphase
#        
#    return Dfield

#Dfield = dphase_loop(N,Dfield,system,Qp,Q,u,X,v,Y)
print(tbox)

[[0.005      0.00500001 0.00500001 ... 0.00502493 0.00502493 0.00502494]]


In [6]:
%%time

Dphase = np.zeros([sample,sample,N],dtype='complex')
Qp = Qprop(system,Q)

u = np.linspace(-amax,amax,sample)
v = u
u,v = np.meshgrid(u,v)

@njit
def dphase_loop(N,Dphase,system,Qp,Q,u,v,prop,tbox):
    for ind in range(N):
        up = u-prop[0,ind]
        vp = v-prop[1,ind]
        rQmult = Qp[0,0]*up**2 + (Qp[0,1]+Qp[1,0])*up*vp + Qp[1,1]*vp**2
        Gphase = (-1j*k)*(tbox[0,ind]+((1/2)*rQmult))
        Dphase[:,:,ind] = Gphase
        
    return Dphase


CPU times: user 1.17 ms, sys: 6.89 ms, total: 8.07 ms
Wall time: 55.6 ms


In [7]:
Dphase = dphase_loop(N,Dphase,system,Qp,Q,u,v,prop,tbox)

In [8]:
Damp = 1/np.sqrt(np.linalg.norm(A+np.matmul(B,Q)))

In [ ]:
Dfield = Damp*ne.evaluate('exp(Dphase)')

In [ ]:
Efield = np.sum(Dfield,axis=2,dtype=complex)

In [ ]:
# Interesting problem when focusing
# When a large number of beamlets are traced, the psf gets spread out
# Small number of beamlets concentrate
# The peak power doesn't appear to change meaningfully, but there is more power spread across a wider area
# Geometrically, the beamlet center rays are all at 0,0
# 
# Since the #beamlets corresponds to their size, it could be that the beamlets are too divergent!!!


plt.figure(figsize=[10,10])
plt.pcolor(u,v,(np.real(Efield*np.conj(Efield))))
#plt.colorbar()